In [1]:
import pandas as pd
import re
import emoji
import string
from emoticon_lib import replace_emoticons_with_text
from deep_translator import (GoogleTranslator)
import html
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions
import numpy as np
from symspellpy import SymSpell, Verbosity
#import sys
#print(sys.executable)
#%pip install wordcloud
#%pip install contractions

In [12]:
df = pd.read_csv('cyberbullying_tweets.csv')

DATA UNDERSTANDING

In [5]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [19]:
df.tail()

,tweet_text,cyberbullying_type
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity
47691,Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...,ethnicity


In [13]:
df.shape

(47692, 2)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [7]:
df.describe()

,tweet_text,cyberbullying_type
count,47692,47692
unique,46017,6
top,RT @sailorhg: the intro for my hardware hackin...,religion
freq,2,7998


In [8]:
df.isna().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [10]:
# Unique values of target variable
df.cyberbullying_type.unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [12]:
# Count of values of target variable
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [18]:
print('Num of Duplicated Rows:', df.duplicated().sum())

Num of Duplicated Rows: 0


DATA CLEANING

In [13]:
# Remove Duplicates
df =df.drop_duplicates(subset='tweet_text', keep="last")

In [14]:
# Expand contraction
df['tweet_text'] = df['tweet_text'].apply(lambda e: contractions.fix(e))
# df.head(10)
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                                          | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words #katandandre, your food was crapilicious! #mkr                                                                       | not_cyberbullying    |
|  1 | Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc                 | not_cyberbullying    |
|  2 | @XochitlSuckkks a classy whore? Or more red velvet cupcakes?                                                                        | not_cyberbullying    |
|  4 | @RudhoeEnglish This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.                             | not_cyberbullying    |
|  5 | @Raja5aab

In [15]:
#Remove html tags
df['tweet_text'] = df['tweet_text'].apply(lambda e: html.unescape(e))

In [16]:
# Remove emoticons
df['tweet_text'] = df['tweet_text'].apply(lambda e: replace_emoticons_with_text(e))

In [17]:
# Remove email 
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"\S+@\S+(?:\.\S+)+", "", e))
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                                          | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words #katandandre, your food was crapilicious! #mkr                                                                       | not_cyberbullying    |
|  1 | Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc                 | not_cyberbullying    |
|  2 | @XochitlSuckkks a classy whore? Or more red velvet cupcakes?                                                                        | not_cyberbullying    |
|  4 | @RudhoeEnglish This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.                             | not_cyberbullying    |
|  5 | @Raja5aab

In [18]:
## Remove RT, #, URL and @: 
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"(RT)|(?:\@|\#|https?|http?|www)\S+", "", e))
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                  | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words  your food was crapilicious!                                                                 | not_cyberbullying    |
|  1 | Why is  so white?                                                                                           | not_cyberbullying    |
|  2 | a classy whore? Or more red velvet cupcakes?                                                                | not_cyberbullying    |
|  4 | This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.                    | not_cyberbullying    |
|  5 | Yes, the test of god is that good or bad or indifferent or weird or whatever, it all proves gods existence. | not_cyberbullying    |
|  6 | Itu sekolah y

In [19]:
# Replace underscore
df['tweet_text'] = df['tweet_text'].apply(lambda e: e.replace('_', ' '))

In [20]:
# Change emojis to text
df['tweet_text'] = df['tweet_text'].apply(lambda e: emoji.demojize(e, language="en"))

In [21]:
# Replace underscore (space) and : (remove)
df['tweet_text'] = df['tweet_text'].apply(lambda e: e.replace('_', ' '))
df['tweet_text'] = df['tweet_text'].apply(lambda e: e.replace(':', ''))

In [22]:
# Word Segmentation  
import wordninja
df['tweet_text'] = df['tweet_text'].apply(lambda x: ' '.join(wordninja.split(x)))
df.head(10)    

,tweet_text,cyberbullying_type
0,In other words your food was crap i licious,not_cyberbullying
1,Why is so white,not_cyberbullying
2,a classy whore Or more red velvet cupcakes,not_cyberbullying
4,This is an IS IS account pretending to be a Ku...,not_cyberbullying
5,Yes the test of god is that good or bad or in ...,not_cyberbullying
6,It u sekolah ya buk an temp at bully Ga j a uh...,not_cyberbullying
7,Karma I hope it bites Kat on the butt She is j...,not_cyberbullying
9,Rebecca Black Drops Out of School Due to Bullying,not_cyberbullying
10,annoyed undecided uneasy or hesitant t co Us Q...,not_cyberbullying
11,The Bully flushes on K D annoyed undecided une...,not_cyberbullying


In [23]:
print(df.head(20).to_markdown())
print(df.tail(20).to_markdown())

|    | tweet_text                                                                                                                                                          | cyberbullying_type   |
|---:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words your food was crap i licious                                                                                                                         | not_cyberbullying    |
|  1 | Why is so white                                                                                                                                                     | not_cyberbullying    |
|  2 | a classy whore Or more red velvet cupcakes                                                                                                                          | not_cyberbullying    |
|  4 | This is an IS

In [24]:
# Remove digits
df['tweet_text'] = df['tweet_text'].replace(to_replace=r'\d', value='', regex=True)

In [25]:
# Drop rows with nan or empty spaces 
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df = df[df['tweet_text'].notna()]

In [26]:
# Remove special characters and digits
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r'[^a-zA-Z ]', '', e))

In [27]:
# Translate
df['tweet_text'] = df['tweet_text'].apply(lambda e: GoogleTranslator(source='auto', target='en').translate(text = e))

In [ ]:
# Remove punctuations
# def remove_punctuations(text):
#     remove_punc = [w for w in text if w not in string.punctuation]
#     remove_punc = ''.join(remove_punc)
#     return ''.join([wrd for wrd in remove_punc.split()])
# df['tweet_text'] = df['tweet_text'].apply(remove_punctuations)


In [ ]:
# Remove extra whitespaces after removing noise and punctuations
def remove_extra_whitespaces(text):
    # Replace multiple consecutive whitespaces with a single space
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()  # Remove leading and trailing whitespaces
df['tweet_text'] = df['tweet_text'].apply(remove_extra_whitespaces)

In [ ]:
# Spelling Correction   SEEMS LIKE NOT WORKING
# symsp = SymSpell()
# symsp.load_dictionary('freq_dictionay_en.txt',term_index=0, count_index=1)
# def correct_spelling(text):
#     suggestions = symsp.lookup_compound(text, max_edit_distance=2)
#     corrected_text = suggestions[0].term if suggestions else text
#     return corrected_text
# df['tweet_text'] = df['tweet_text'].apply(correct_spelling)


In [ ]:
# Normalize Words / Remove repeating words with 1 or  (use regex) 
def remove_words_with_one_char_or_repeating_chars(text):   
    text = re.sub(r'\b\w\b', '', text)   # Remove words with 1 character
    text = re.sub(r'\b\w*(\w)\1{2,}\w*\b', '', text)   # Remove words with 3 or more repeating characters
    return text
df['tweet_text'] = df['tweet_text'].apply(remove_words_with_one_char_or_repeating_chars)
print(df)

In [ ]:
# Convert to lower case
df['tweet_text'] = df['tweet_text'].str.lower()

In [ ]:
# Tokenization
df['tweet_text'] = df.apply(lambda row: word_tokenize(row['tweet_text']), axis=1)
print(df.head(10).to_markdown())

# Remove stopwords
stopword = stopwords.words('english')
df['tweet_text'] = df['tweet_text'].apply(lambda e: ' '.join([word for word in e.split() if word not in (stopword)]))
print(df.head(10).to_markdown())

In [11]:
stopword = stopwords.words('english')
stopword

def is_stopword(word):
    return word.lower() in stopword

line = "I like i example sentence to test stopwords"
words = line.split()

for word in words:
    if is_stopword(word):
        print(f"'{word}' is a stopword.")
    else:
        print(f"'{word}' is not a stopword.")


'I' is a stopword.
'like' is not a stopword.
'i' is a stopword.
'example' is not a stopword.
'sentence' is not a stopword.
'to' is a stopword.
'test' is not a stopword.
'stopwords' is not a stopword.


In [ ]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df["tweet_text"] = df["tweet_text"].apply(lambda e: lemmatize_words(e))

In [ ]:
# Save processed dataset into new csv file
df.to_csv('preprocessed_cyberbullying_tweet.csv', index=False)